In [ ]:
  #구글드라이브 연동
from google.colab import drive    
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
  !pip install mxnet
  !pip install gluonnlp pandas tqdm
  !pip install sentencepiece
  !pip install transformers==3.0.2


     |████████████████████████████████| 47.3 MB 1.9 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595736 sha256=5b513dcd733e72c83a175af4d0bd0952ece5ea6318869746693d33572b3a6aa7
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.3 MB/s 
     |████████████████████████████████| 769 kB 5.3 MB/s 
     |████████████████████████████████| 3.0 MB 29.4 MB/s 
     |████████████████████████████████| 895 kB 14.8 MB/s 


In [ ]:
!pip install torch

In [ ]:
# pip install torch==1.10.0+cu113 torchvision==0.11.1+cu113 torchaudio===0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

In [ ]:
# #깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-mxoprvwy
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-mxoprvwy
     |████████████████████████████████| 132 kB 5.2 MB/s 
     |████████████████████████████████| 4.5 MB 33.3 MB/s 
     |████████████████████████████████| 3.5 MB 35.2 MB/s 
     |████████████████████████████████| 596 kB 18.7 MB/s 
     |████████████████████████████████| 6.8 MB 33.1 MB/s 
     |████████████████████████████████| 67 kB 2.5 MB/s 
     |████████████████████████████████| 8.5 MB 31.2 MB/s 
     |████████████████████████████████| 79 kB 4.6 MB/s 
     |████████████████████████████████| 127 kB 32.8 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15449 sha256=9df13c550c404ecc2e9c2ee5a26f99db2bbda45f52fcd45ba3b9c7b194a85c53
  Stored in directory: /tmp/pip-ephem-wheel-cache-g3jaw58w/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd05

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용
device = torch.device("cuda")

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
import pandas as pd
review_data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/mutbly/shopping_review.xlsx')

In [ ]:
review_data[1000:1100]

,label,contents
1000,0,기대 안했는데 좋으네요 ~
1001,0,기대 이상으로 시원하네요~재주문가요
1002,0,기대 이상이에요 숫자도 넉넉하고 생각했던 것보다 괜찮아요. 숫자 뜯다가 힘줘서 부러...
1003,0,기대 이하예요
1004,0,기대가너무컸을까요... 작은크기에 가시까지...
...,...,...
1095,0,"깔끔하고 고급져요. 기대이상입니다. 잘 사용하겠습니다. 다만, 강아지가 긁으니 실밥..."
1096,0,깔끔하고 기능별로 나눠서 쓸수있어서 좋네요 받아서 만족하고 있어요
1097,0,깔끔하고 길어서 좋넹ᆢ
1098,0,깔끔하고 깨끗하게 시공이 잘되네요


In [ ]:
import re
 
def cleanText(readData):
 
    #텍스트에 포함되어 있는 특수 문자 제거
 
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', readData)
 
    return text

 
txt = "안녕!!~~  ㅋㅋ"
 
thisdata = cleanText(txt)

print(thisdata)

안녕  ㅋㅋ


In [ ]:
review_data['contents'] = review_data['contents'].apply(cleanText)

In [ ]:
review_data[200:230]

,label,contents
200,0,가격대비 별로에요
201,0,가격대비 별로입니다
202,0,가격대비 불만족함
203,0,가격대비 상품퀄은떨어지네요 배송이나 포장은잘되서옵니다
204,0,가격대비 생각만큼 괜찮아여 그이상은 아니구요
205,0,가격대비 성능 괜찮네요 신규 사무실 프린트용인데 괜찮습니다
206,0,가격대비 쓸만한거같아여
207,0,가격대비 아주 괜찮아요
208,0,가격대비 아주 만족해요 잘쓸게요
209,0,가격대비 양이너무작지만ㅜㅜ바르면좋아지길 기대해요


In [ ]:
review_data.loc[(review_data['label'] == "0"), 'label'] = 0  #리뷰
review_data.loc[(review_data['label'] == "1"), 'label'] = 1  #욕설
review_data.loc[(review_data['label'] == "2"), 'label'] = 2  #욕설 문장

data_list = []
for q, label in zip(review_data['contents'], review_data['label'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
review_data[200:230]

,label,contents
200,0,가격대비 별로에요
201,0,가격대비 별로입니다
202,0,가격대비 불만족함
203,0,가격대비 상품퀄은떨어지네요 배송이나 포장은잘되서옵니다
204,0,가격대비 생각만큼 괜찮아여 그이상은 아니구요
205,0,가격대비 성능 괜찮네요 신규 사무실 프린트용인데 괜찮습니다
206,0,가격대비 쓸만한거같아여
207,0,가격대비 아주 괜찮아요
208,0,가격대비 아주 만족해요 잘쓸게요
209,0,가격대비 양이너무작지만ㅜㅜ바르면좋아지길 기대해요


In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

10399
3467


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
data_train[0]

(array([   2, 5112, 5673, 5712,    3,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(5, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=1)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=1)

5.KoBERT 학습모델 만들기

학습시킬 KoBERT 모델을 만들어야 하는데, 아래 코드에서 다중분류할 클래스 수 만큼 num_classes 변수를 수정해주어야 한다. 이번 프로젝트에서는 7가지의 class를 분류하기 때문에 7로 입력해주었다.

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,   ##클래스 수 조정
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


6.KoBERT 모델 학습시키기

학습 데이터셋과 학습 모델 준비가 다 끝났다면 이제 아래 코드 실행을 통해 KoBERT 모델을 학습시켜준다.

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/163 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.160536766052246 train acc 0.328125
epoch 1 train acc 0.8703802196714823


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/55 [00:00<?, ?it/s]

...

epoch 4 test acc 0.9807334710743801


  0%|          | 0/163 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.007331680506467819 train acc 1.0
epoch 5 train acc 0.9982745398773006


  0%|          | 0/55 [00:00<?, ?it/s]

epoch 5 test acc 0.9807334710743801


In [ ]:
path = F"/content/drive/MyDrive/mutblymodel2.pth" 

In [ ]:
model.load_state_dict(torch.load(path, map_location=device) , strict=False)

<All keys matched successfully>